### Tokenization:
Tokenization is the process of taking text input and breaking them down into smaller inputs: characters, words, subwords.
These are then mapped to IDs, and form the model's vocabulary, the set of all possible tokens.

This notebook aims to implement a tokenizer. After this, I plan to develop an embedding model. I have some novel ideas I want to try out too...

A few important observations: 
* the same concept might have 2+ different token mapping depending on the context: lower case, upper case, in the beginning of the sentence, at the end of the sentence, etc.

* non-English languages end up having shorter tokens because there is less data from them ---> more tokens/more chunks for the same concepts.  This is particularly bad for context windows, in self-attention since there are more tokens in a simple sentence.

* for coding, indentation causes ineficiencies stemming from wasteful use of the context window using one token per space.

A good tokenizer can decrease the number of tokens per sentence while not increasing the vocabulary too much.

In [11]:
# Accessing the Unicode code points:
ord('a'), ord('あ'), ord('🤔'), ord('青')

(97, 12354, 129300, 38738)

In [12]:
# Encodings translate the unicode point to a byte string -> UTF-8 encodes it to 8 bytes. 
list("What stands in the way becomes the way.".encode('utf-8'))
# Using UTF is inadequate, because the words are mapped to many bytes, so the context used would be too large, 
# and the byte-wise prediction too myopic.

[87,
 104,
 97,
 116,
 32,
 115,
 116,
 97,
 110,
 100,
 115,
 32,
 105,
 110,
 32,
 116,
 104,
 101,
 32,
 119,
 97,
 121,
 32,
 98,
 101,
 99,
 111,
 109,
 101,
 115,
 32,
 116,
 104,
 101,
 32,
 119,
 97,
 121,
 46]

#### BPE: Byte-Pair Encoding

BPE finds the pair of tokens that occur most frequently, iteratively, appending this token to the vocab.

aaabdaaabac

aa is mapped into Z.
Then, the sentence becomes ZabdZabac. ab occurs most frequently, so it turns into Y:

ZYdZYac

ZY appears most frequently, mapping it into X：

XdXac

The most frequent pair only appears once, so the algorithm terminates.
* X=ZY
* Y=ab
* Z=aa

The resulting sequence is compressed.


In [53]:
# Implementation:
text = 'あ'
tokens = text.encode("utf-8") # get the bytes
tokens = list(map(int,tokens)) # convert them into a list of 0..255
print(tokens)

[227, 129, 130]


In [43]:
# Implementation:
text= "When Zarathustra had spoken these words, he again looked at the people, and was silent. “There they stand,” said he to his heart; “there they laugh: they understand me not; I am not the mouth for these ears. Must one first batter their ears, that they may learn to hear with their eyes? Must one clatter like kettledrums and penitential preachers? Or do they only believe the stammerer?They have something whereof they are proud. What do they call it, that which maketh them proud? Culture, they call it; it distinguisheth them from the goatherds. They dislike, therefore, to hear of ‘contempt’ of themselves. So I will appeal to their pride. I will speak unto them of the most contemptible thing: that, however, is THE LAST MAN!” And thus spake Zarathustra unto the people: It is time for man to fix his goal. It is time for man to plant the germ of his highest hope. Still is his soil rich enough for it. But that soil will one day be poor and exhausted, and no lofty tree will any longer be able to grow thereon. Alas! there cometh the time when man will no longer launch the arrow of his longing beyond man—and the string of his bow will have unlearned to whizz! I tell you: one must still have chaos in one, to give birth to a dancing star. I tell you: ye have still chaos in you. Alas! There cometh the time when man will no longer give birth to any star. Alas! There cometh the time of the most despicable man, who can no longer despise himself. Lo! I show you THE LAST MAN. “What is love? What is creation? What is longing? What is a star?”—so asketh the last man and blinketh. The earth hath then become small, and on it there hoppeth the last man who maketh everything small. His species is ineradicable like that of the ground-flea; the last man liveth longest. “We have discovered happiness”—say the last men, and blink thereby. They have left the regions where it is hard to live; for they need warmth. One still loveth one’s neighbour and rubbeth against him; for one needeth warmth. Turning ill and being distrustful, they consider sinful: they walk warily. He is a fool who still stumbleth over stones or men! A little poison now and then: that maketh pleasant dreams. And much poison at last for a pleasant death. One still worketh, for work is a pastime. But one is careful lest the pastime should hurt one. One no longer becometh poor or rich; both are too burdensome. Who still wanteth to rule? Who still wanteth to obey? Both are too burdensome. No shepherd, and one herd! Every one wanteth the same; every one is equal: he who hath other sentiments goeth voluntarily into the madhouse. “Formerly all the world was insane,”—say the subtlest of them, and blink thereby. They are clever and know all that hath happened: so there is no end to their raillery. People still fall out, but are soon reconciled—otherwise it spoileth their stomachs. They have their little pleasures for the day, and their little pleasures for the night, but they have a regard for health. “We have discovered happiness,”—say the last men, and blink thereby.— And here ended the first discourse of Zarathustra, which is also called “The Prologue”: for at this point the shouting and mirth of the multitude interrupted him. “Give us this last man, O Zarathustra,”—they called out—“make us into these last men! Then will we make thee a present of the Superman!” And all the people exulted and smacked their lips. Zarathustra, however, turned sad, and said to his heart: “They understand me not: I am not the mouth for these ears. Too long, perhaps, have I lived in the mountains; too much have I hearkened unto the brooks and trees: now do I speak unto them as unto the goatherds. Calm is my soul, and clear, like the mountains in the morning. But they think me cold, and a mocker with terrible jests. And now do they look at me and laugh: and while they laugh they hate me too. There is ice in their laughter.” Then, however, something happened which made every mouth mute and every eye fixed. In the meantime, of course, the rope-dancer had commenced his performance: he had come out at a little door, and was going along the rope which was stretched between two towers, so that it hung above the market-place and the people. When he was just midway across, the little door opened once more, and a gaudily-dressed fellow like a buffoon sprang out, and went rapidly after the first one. “Go on, halt-foot,” cried his frightful voice, “go on, lazy-bones, interloper, sallow-face!—lest I tickle thee with my heel! What dost thou here between the towers? In the tower is the place for thee, thou shouldst be locked up; to one better than thyself thou blockest the way!”—And with every word he came nearer and nearer the first one. When, however, he was but a step behind, there happened the frightful thing which made every mouth mute and every eye fixed—he uttered a yell like a devil, and jumped over the other who was in his way. The latter, however, when he thus saw his rival triumph, lost at the same time his head and his footing on the rope; he threw his pole away, and shot downwards faster than it, like an eddy of arms and legs, into the depth. The market-place and the people were like the sea when the storm cometh on: they all flew apart and in disorder, especially where the body was about to fall. Zarathustra, however, remained standing, and just beside him fell the body, badly injured and disfigured, but not yet dead. After a while consciousness returned to the shattered man, and he saw Zarathustra kneeling beside him. “What art thou doing there?” said he at last, “I knew long ago that the devil would trip me up. Now he draggeth me to hell: wilt thou prevent him? “On mine honour, my friend,” answered Zarathustra, “there is nothing of all that whereof thou speakest: there is no devil and no hell. Thy soul will be dead even sooner than thy body: fear, therefore, nothing any more!” The man looked up distrustfully. “If thou speakest the truth,” said he, “I lose nothing when I lose my life. I am not much more than an animal which hath been taught to dance by blows and scanty fare.” “Not at all,” said Zarathustra, “thou hast made danger thy calling; therein there is nothing contemptible. Now thou perishest by thy calling: therefore will I bury thee with mine own hands.” When Zarathustra had said this the dying one did not reply further; but he moved his hand as if he sought the hand of Zarathustra in gratitude.🔥"
tokens = text.encode("utf-8") # get the bytes
tokens = list(map(int,tokens)) # convert them into a list of 0..255
print(tokens)

[87, 104, 101, 110, 32, 90, 97, 114, 97, 116, 104, 117, 115, 116, 114, 97, 32, 104, 97, 100, 32, 115, 112, 111, 107, 101, 110, 32, 116, 104, 101, 115, 101, 32, 119, 111, 114, 100, 115, 44, 32, 104, 101, 32, 97, 103, 97, 105, 110, 32, 108, 111, 111, 107, 101, 100, 32, 97, 116, 32, 116, 104, 101, 32, 112, 101, 111, 112, 108, 101, 44, 32, 97, 110, 100, 32, 119, 97, 115, 32, 115, 105, 108, 101, 110, 116, 46, 32, 226, 128, 156, 84, 104, 101, 114, 101, 32, 116, 104, 101, 121, 32, 115, 116, 97, 110, 100, 44, 226, 128, 157, 32, 115, 97, 105, 100, 32, 104, 101, 32, 116, 111, 32, 104, 105, 115, 32, 104, 101, 97, 114, 116, 59, 32, 226, 128, 156, 116, 104, 101, 114, 101, 32, 116, 104, 101, 121, 32, 108, 97, 117, 103, 104, 58, 32, 116, 104, 101, 121, 32, 117, 110, 100, 101, 114, 115, 116, 97, 110, 100, 32, 109, 101, 32, 110, 111, 116, 59, 32, 73, 32, 97, 109, 32, 110, 111, 116, 32, 116, 104, 101, 32, 109, 111, 117, 116, 104, 32, 102, 111, 114, 32, 116, 104, 101, 115, 101, 32, 101, 97, 114, 115, 46,

In [44]:
len(text), len(tokens) # some characters are mapped into more than one byte, like the emoji

(6436, 6547)

In [45]:
def get_stats (text):
    counts = {}
    for pair in zip(text,text[1:]):
        counts[pair] = counts.get(pair,0) +1
    return counts

stats = get_stats(tokens)
print(sorted(((v,k) for k,v in stats.items()), reverse= True))

[(234, (116, 104)), (226, (101, 32)), (203, (32, 116)), (197, (104, 101)), (104, (100, 32)), (104, (32, 97)), (99, (101, 114)), (94, (116, 32)), (92, (32, 104)), (89, (97, 110)), (78, (115, 116)), (71, (115, 32)), (71, (44, 32)), (70, (114, 101)), (69, (32, 115)), (65, (32, 119)), (64, (32, 109)), (63, (110, 100)), (62, (121, 32)), (62, (111, 32)), (60, (105, 110)), (57, (104, 97)), (57, (32, 108)), (56, (114, 32)), (56, (105, 115)), (54, (226, 128)), (54, (104, 32)), (54, (46, 32)), (52, (111, 110)), (52, (32, 111)), (51, (110, 32)), (50, (97, 116)), (50, (32, 105)), (49, (97, 114)), (47, (108, 108)), (45, (110, 101)), (45, (108, 101)), (45, (108, 32)), (45, (104, 105)), (44, (118, 101)), (43, (32, 98)), (42, (101, 110)), (41, (111, 117)), (40, (110, 103)), (40, (101, 100)), (39, (116, 111)), (39, (109, 101)), (36, (111, 114)), (36, (32, 102)), (36, (32, 100)), (35, (105, 108)), (35, (101, 116)), (35, (101, 97)), (32, (110, 116)), (32, (109, 97)), (32, (101, 115)), (31, (104, 111)), (

In [46]:
chr(101), chr(32) # the most common pair

('e', ' ')

In [47]:
# creating new tokens, iterating
top_pair = max(stats, key=stats.get)
top_pair

(116, 104)

In [48]:
def merge(ids, pair, idx):
    newids =[]
    i = 0
    while i <len(ids):
        if i< len(ids) -1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i=i+2
        else: 
            newids.append(ids[i])
            i= i +1
    return newids

print(merge([1,2,2,4,5,6],(2,4),120)) # replaces (2,4) with 120

[1, 2, 120, 5, 6]


In [49]:
tokens2 = merge(tokens, top_pair, 256)
print(tokens2) 
len(tokens), len(tokens2) # reduced size

[87, 104, 101, 110, 32, 90, 97, 114, 97, 256, 117, 115, 116, 114, 97, 32, 104, 97, 100, 32, 115, 112, 111, 107, 101, 110, 32, 256, 101, 115, 101, 32, 119, 111, 114, 100, 115, 44, 32, 104, 101, 32, 97, 103, 97, 105, 110, 32, 108, 111, 111, 107, 101, 100, 32, 97, 116, 32, 256, 101, 32, 112, 101, 111, 112, 108, 101, 44, 32, 97, 110, 100, 32, 119, 97, 115, 32, 115, 105, 108, 101, 110, 116, 46, 32, 226, 128, 156, 84, 104, 101, 114, 101, 32, 256, 101, 121, 32, 115, 116, 97, 110, 100, 44, 226, 128, 157, 32, 115, 97, 105, 100, 32, 104, 101, 32, 116, 111, 32, 104, 105, 115, 32, 104, 101, 97, 114, 116, 59, 32, 226, 128, 156, 256, 101, 114, 101, 32, 256, 101, 121, 32, 108, 97, 117, 103, 104, 58, 32, 256, 101, 121, 32, 117, 110, 100, 101, 114, 115, 116, 97, 110, 100, 32, 109, 101, 32, 110, 111, 116, 59, 32, 73, 32, 97, 109, 32, 110, 111, 116, 32, 256, 101, 32, 109, 111, 117, 256, 32, 102, 111, 114, 32, 256, 101, 115, 101, 32, 101, 97, 114, 115, 46, 32, 77, 117, 115, 116, 32, 111, 110, 101, 32, 102

(6547, 6313)

In [50]:
# running this for the whole text:
# more steps -> larger vocabulary, shorter sequence trade off
# this is a hyperparameter to be tuned.

vocab_size = 276 # desired vocab size
num_merges = vocab_size -256
ids = list(tokens)

merges = {}
for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = 256 + i
    print(f"merging {pair} into token {idx}")
    ids = merge(ids,pair,idx)
    merges[pair] = idx

merging (116, 104) into token 256
merging (101, 32) into token 257
merging (32, 256) into token 258
merging (101, 114) into token 259
merging (100, 32) into token 260
merging (97, 110) into token 261
merging (115, 116) into token 262
merging (115, 32) into token 263
merging (44, 32) into token 264
merging (105, 110) into token 265
merging (258, 257) into token 266
merging (121, 32) into token 267
merging (46, 32) into token 268
merging (226, 128) into token 269
merging (111, 110) into token 270
merging (97, 114) into token 271
merging (108, 108) into token 272
merging (116, 32) into token 273
merging (111, 32) into token 274
merging (101, 110) into token 275


In [51]:
# compression: how was the original text compressed? The vocab increased by 20.
print("tokens len:", len(tokens))
print("ids len:", len(ids))
print(f"compression ratio: {len(tokens)/len(ids):.2f}x")

tokens len: 6547
ids len: 4892
compression ratio: 1.34x


The tokenizer is completly isolated from the actual LLM. It has its own dataset. The LLM only deals with tokens.

So the tokenizer is essentialy a pre-processing/translation layer 

In [67]:
# creating the encoder/decoder: tokens to and from text
vocab = {idx: bytes([idx]) for idx in range(256)} # maps token id to the bytes object for the token
for (p0,p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1] #concatenates bytes objects

def decode(ids):
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode(encoding= "utf-8",errors = "replace")
    return text

print(decode([40,103,103,41]))

(gg)


In [70]:
def encode(text):
    tokens = list(text.encode("utf-8")) # raw bytes
    # unroll merges top to bottom
    while True:
        stats = get_stats(tokens)
        pair = min(stats, key= lambda p: merges.get(p,float("inf")))
        if pair not in merges:
            break # no more merges
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

print(encode("I tell you: one must still have chaos in one, to give birth to a dancing star."))

[73, 32, 116, 101, 272, 32, 121, 111, 117, 58, 32, 270, 257, 109, 117, 262, 32, 262, 105, 272, 32, 104, 97, 118, 257, 99, 104, 97, 111, 263, 265, 32, 270, 101, 264, 116, 274, 103, 105, 118, 257, 98, 105, 114, 256, 32, 116, 274, 97, 32, 100, 261, 99, 265, 103, 32, 262, 271, 46]


In [75]:
print(decode(encode("hello"))) 

hello


In [76]:
text2 = decode(encode(text))
print(text==text2)

True


BPE is suboptimal in many cases.